In [48]:
import csv
import numpy as np
import pandas as pd

In [49]:
train_df = pd.read_csv("../data/regression/train.csv")

test_df = pd.read_csv("../data/regression/test.csv")

sample_submission = pd.read_csv("../data/regression/sample_submission.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

Train shape: (41257, 20)
Test shape: (13753, 19)


In [51]:
# Merge train and test data for easier feature engineering

train_df["is_train"] = 1
test_df["is_train"] = 0
test_df["co2"] = np.nan

data_df = pd.concat([train_df, test_df], sort=False)

In [52]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55010 entries, 0 to 13752
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                55010 non-null  int64  
 1   brand             55010 non-null  object 
 2   model             55010 non-null  object 
 3   car_class         55010 non-null  object 
 4   range             55010 non-null  object 
 5   fuel_type         55010 non-null  object 
 6   hybrid            55010 non-null  object 
 7   max_power         55010 non-null  float64
 8   grbx_type_ratios  55010 non-null  object 
 9   weight_min        55010 non-null  int64  
 10  weight_max        55010 non-null  int64  
 11  urb_cons          55001 non-null  float64
 12  exturb_cons       55001 non-null  float64
 13  overall_cons      55010 non-null  float64
 14  co                54886 non-null  float64
 15  hc                9773 non-null   float64
 16  nox               54886 non-null  float64
 17

In [54]:
data_df.head()

,id,brand,model,car_class,range,fuel_type,hybrid,max_power,grbx_type_ratios,weight_min,...,urb_cons,exturb_cons,overall_cons,co,hc,nox,hcnox,ptcl,co2,is_train
0,0,MERCEDES,COMBI 110 CDI,MINIBUS,MOY-INFER,GO,non,70.0,M 6,1976,...,9.1,6.4,7.4,0.083,NaN,0.229,0.250,0.001,195.0,1
1,1,MERCEDES,VIANO 2.0 CDI,MINIBUS,MOY-SUPER,GO,non,100.0,A 5,2186,...,10.2,7.0,8.2,0.078,NaN,0.224,0.233,0.001,216.0,1
2,2,MERCEDES,SPRINTER COMBI 319 CDI,MINIBUS,MOY-INFER,GO,non,140.0,A 5,2586,...,12.5,9.0,10.3,0.067,0.014,1.846,NaN,0.002,272.0,1
3,3,RENAULT,MEGANE Coupé EnergyTCe (115ch) eco2,COUPE,MOY-INFER,ES,non,85.0,M 6,1280,...,6.4,4.6,5.3,0.167,0.039,0.039,NaN,0.001,119.0,1
4,4,MERCEDES,COMBI 116 CDI,MINIBUS,MOY-INFER,GO,non,120.0,A 5,2356,...,10.1,6.9,8.1,0.042,NaN,0.190,0.201,0.001,214.0,1


In [55]:
data_df['model'].value_counts()

model
VIANO 2.2 CDI                                            5874
VIANO 2.0 CDI                                            3903
COMBI 116 CDI                                            3754
COMBI 113 CDI                                            2620
VIANO 3.0 CDI                                            1608
                                                         ... 
A6 V6 3.0 TDI CLEAN DIESEL (245ch) QUATTRO S TRONIC 7       1
NOUVELLE AURIS (132ch) VVT-i (Style)                        1
MULTIVAN CONFORTLINE LONG 2.0 TDI (140ch) BlueMotion        1
Thema 3.0 Mulitjet II (190ch)                               1
PANAMERA 4S                                                 1
Name: count, Length: 3817, dtype: int64

In [57]:
data_df['hc'] = data_df['hc'].fillna(data_df['hcnox']-data_df['nox'])
data_df = data_df.drop(columns=['hcnox'])

In [58]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55010 entries, 0 to 13752
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                55010 non-null  int64  
 1   brand             55010 non-null  object 
 2   model             55010 non-null  object 
 3   car_class         55010 non-null  object 
 4   range             55010 non-null  object 
 5   fuel_type         55010 non-null  object 
 6   hybrid            55010 non-null  object 
 7   max_power         55010 non-null  float64
 8   grbx_type_ratios  55010 non-null  object 
 9   weight_min        55010 non-null  int64  
 10  weight_max        55010 non-null  int64  
 11  urb_cons          55001 non-null  float64
 12  exturb_cons       55001 non-null  float64
 13  overall_cons      55010 non-null  float64
 14  co                54886 non-null  float64
 15  hc                54886 non-null  float64
 16  nox               54886 non-null  float64
 17

In [59]:
data_df = data_df.dropna(subset=['co'])
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54886 entries, 0 to 13752
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                54886 non-null  int64  
 1   brand             54886 non-null  object 
 2   model             54886 non-null  object 
 3   car_class         54886 non-null  object 
 4   range             54886 non-null  object 
 5   fuel_type         54886 non-null  object 
 6   hybrid            54886 non-null  object 
 7   max_power         54886 non-null  float64
 8   grbx_type_ratios  54886 non-null  object 
 9   weight_min        54886 non-null  int64  
 10  weight_max        54886 non-null  int64  
 11  urb_cons          54877 non-null  float64
 12  exturb_cons       54877 non-null  float64
 13  overall_cons      54886 non-null  float64
 14  co                54886 non-null  float64
 15  hc                54886 non-null  float64
 16  nox               54886 non-null  float64
 17

In [ ]:
data_df.head()